In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import concat, lit

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Constructors_pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# load the results, drivers and constructors tables in dataframes
gamesDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data_de2023_2124849/nba-dataset/games.csv")
gamesDetailsDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data_de2023_2124849/nba-dataset/games_details.csv")
playersDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data_de2023_2124849/nba-dataset/players.csv")
rankingDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data_de2023_2124849/nba-dataset/ranking.csv")
teamsDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data_de2023_2124849/nba-dataset/teams.csv")


gamesDF.show(5)
gamesDetailsDF.show(5)
playersDF.show(5)
rankingDF.show(5)
teamsDF.show(5)

+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+-----------+------------+--------+--------+------------+--------+-----------+-----------+------------+--------+--------+--------------+
|GAME_DATE_EST| GAME_ID|GAME_STATUS_TEXT|HOME_TEAM_ID|VISITOR_TEAM_ID|SEASON|TEAM_ID_home|PTS_home|FG_PCT_home|FT_PCT_home|FG3_PCT_home|AST_home|REB_home|TEAM_ID_away|PTS_away|FG_PCT_away|FT_PCT_away|FG3_PCT_away|AST_away|REB_away|HOME_TEAM_WINS|
+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+-----------+------------+--------+--------+------------+--------+-----------+-----------+------------+--------+--------+--------------+
|   2022-12-22|22200477|           Final|  1610612740|     1610612759|  2022|  1610612740|     126|      0.484|      0.926|       0.382|      25|      46|  1610612759|     117|      0.478|      0.815|       0.321|      23|      44|             1|
|   2022-12-

In [22]:
### lets calculate which teams scored the most points
### We can also calculaete whats the percantage of home wins
### First lets drop the columns which we do not understand (lack of description on Kaggle and I cannot understand it from the name of the column) :/ 
filteredGAMESDF = gamesDF.select("GAME_DATE_EST", "GAME_ID", "GAME_STATUS_TEXT", "HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "TEAM_ID_home", "PTS_home", "FG_PCT_home", "TEAM_ID_away", "PTS_away", "FG_PCT_away", "HOME_TEAM_WINS")
filteredGAMESDetailsDF = gamesDetailsDF.select("TEAM_ID","TEAM_ABBREVIATION", "PLAYER_ID", "PLAYER_NAME", "MIN", "AST", "STL", "PF")
## we keep players as it is
filteredTeamsDF = teamsDF.select("LEAGUE_ID", "TEAM_ID", "MIN_YEAR", "MAX_YEAR", "ABBREVIATION", "NICKNAME", "HEADCOACH")
filteredRankingDF = rankingDF.select("TEAM_ID", "LEAGUE_ID", "SEASON_ID", "CONFERENCE", "TEAM", "G", "W", "L", "W_PCT")

filteredGAMESDF.show(5)
filteredGAMESDetailsDF.show(5)
print('Here are filtered rankings')
filteredRankingDF.show(5)
print('HEre are filtered teams')
filteredTeamsDF.show(5)


+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+------------+--------+-----------+--------------+
|GAME_DATE_EST| GAME_ID|GAME_STATUS_TEXT|HOME_TEAM_ID|VISITOR_TEAM_ID|SEASON|TEAM_ID_home|PTS_home|FG_PCT_home|TEAM_ID_away|PTS_away|FG_PCT_away|HOME_TEAM_WINS|
+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+------------+--------+-----------+--------------+
|   2022-12-22|22200477|           Final|  1610612740|     1610612759|  2022|  1610612740|     126|      0.484|  1610612759|     117|      0.478|             1|
|   2022-12-22|22200478|           Final|  1610612762|     1610612764|  2022|  1610612762|     120|      0.488|  1610612764|     112|      0.561|             1|
|   2022-12-21|22200466|           Final|  1610612739|     1610612749|  2022|  1610612739|     114|      0.482|  1610612749|     106|       0.47|             1|
|   2022-12-21|22200467|          

In [41]:
from pyspark.sql.functions import col  # Import the 'col' function

# Grouping by TEAM_ID and summing up the points scored both as home and away
total_points = filteredGAMESDF.groupBy('TEAM_ID_home').agg({'PTS_home': 'sum'}) \
            .withColumnRenamed('sum(PTS_home)', 'total_points_home') \
            .join(filteredGAMESDF.groupBy('TEAM_ID_away').agg({'PTS_away': 'sum'}) \
            .withColumnRenamed('sum(PTS_away)', 'total_points_away'), 
            col('TEAM_ID_home') == col('TEAM_ID_away')) \
            .withColumn('total_points', col('total_points_home') + col('total_points_away')) \
            .select('TEAM_ID_home', 'total_points')

total_points.show(5)

# Joining with teams_df to get team names
team_total_points = total_points.join(filteredTeamsDF, total_points.TEAM_ID_home == filteredTeamsDF.TEAM_ID) \
                    .select('TEAM_ID_home', 'NICKNAME', 'total_points') \
                    .orderBy(col('total_points').desc())

# Showing the team with the most total points
team_total_points.show(10)

team_total_points_conference = team_total_points.join(filteredRankingDF, 
                                team_total_points.TEAM_ID_home == filteredRankingDF.TEAM_ID) \
                                .select(team_total_points.TEAM_ID_home, 
                                         team_total_points.NICKNAME, 
                                         team_total_points.total_points, 
                                         filteredRankingDF.CONFERENCE).orderBy(col('total_points').desc())


# Showing the team with the most total points
withoutduplicates = team_total_points_conference.dropDuplicates()

withoutduplicates_order = withoutduplicates.orderBy(col('total_points').desc())

withoutduplicates_order.show(10)


+------------+------------+
|TEAM_ID_home|total_points|
+------------+------------+
|  1610612755|    175560.0|
|  1610612753|    174421.0|
|  1610612759|    190888.0|
|  1610612740|    173776.0|
|  1610612758|    174952.0|
+------------+------------+
only showing top 5 rows

+------------+---------+------------+
|TEAM_ID_home| NICKNAME|total_points|
+------------+---------+------------+
|  1610612744| Warriors|    195235.0|
|  1610612738|  Celtics|    191453.0|
|  1610612759|    Spurs|    190888.0|
|  1610612748|     Heat|    189712.0|
|  1610612743|  Nuggets|    189581.0|
|  1610612747|   Lakers|    189517.0|
|  1610612756|     Suns|    188130.0|
|  1610612742|Mavericks|    187295.0|
|  1610612745|  Rockets|    185799.0|
|  1610612760|  Thunder|    184942.0|
+------------+---------+------------+
only showing top 10 rows

+------------+---------+------------+----------+
|TEAM_ID_home| NICKNAME|total_points|CONFERENCE|
+------------+---------+------------+----------+
|  1610612744| War

In [68]:
from pyspark.sql.window import Window
from pyspark.sql.functions import max, dense_rank

## Now we will ask dense ranking and later we will choose top 3 for every conference

rankedTeamsWindow = Window.partitionBy("CONFERENCE").orderBy(col("total_points").desc())
denseRankPoints = dense_rank().over(rankedTeamsWindow)
dfRankedTeams = withoutduplicates_order.select(col("TEAM_ID_home"), col('NICKNAME'), col("CONFERENCE"), col("total_points").alias("total_points_by_team"), denseRankPoints.alias("denseRankTotalPoints"))
dfRankedTeams.show()

+------------+---------+----------+--------------------+--------------------+
|TEAM_ID_home| NICKNAME|CONFERENCE|total_points_by_team|denseRankTotalPoints|
+------------+---------+----------+--------------------+--------------------+
|  1610612738|  Celtics|      East|            191453.0|                   1|
|  1610612748|     Heat|      East|            189712.0|                   2|
|  1610612739|Cavaliers|      East|            182099.0|                   3|
|  1610612761|  Raptors|      East|            181520.0|                   4|
|  1610612749|    Bucks|      East|            181416.0|                   5|
|  1610612754|   Pacers|      East|            180054.0|                   6|
|  1610612737|    Hawks|      East|            180039.0|                   7|
|  1610612764|  Wizards|      East|            177752.0|                   8|
|  1610612741|    Bulls|      East|            176116.0|                   9|
|  1610612751|     Nets|      East|            175688.0|        

In [69]:
# Keep the best 3 for every conference 
dfBestTeams = dfRankedTeams.where((col("denseRankTotalPoints")==1) | (col("denseRankTotalPoints")==2) | (col("denseRankTotalPoints")==3))
dfBestTeams = dfBestTeams.withColumnRenamed("TEAM_ID_home", "TEAM_ID")

dfBestTeams.show()

+------------+---------+----------+--------------------+--------------------+
|TEAM_ID_home| NICKNAME|CONFERENCE|total_points_by_team|denseRankTotalPoints|
+------------+---------+----------+--------------------+--------------------+
|  1610612738|  Celtics|      East|            191453.0|                   1|
|  1610612748|     Heat|      East|            189712.0|                   2|
|  1610612739|Cavaliers|      East|            182099.0|                   3|
|  1610612744| Warriors|      West|            195235.0|                   1|
|  1610612759|    Spurs|      West|            190888.0|                   2|
|  1610612743|  Nuggets|      West|            189581.0|                   3|
+------------+---------+----------+--------------------+--------------------+

+----------+---------+----------+--------------------+--------------------+
|   TEAM_ID| NICKNAME|CONFERENCE|total_points_by_team|denseRankTotalPoints|
+----------+---------+----------+--------------------+-------------

In [79]:
# Join result with 
filteredTeamsDF = filteredTeamsDF.drop('NICKNAME')
filteredTeamsDF.show(10)
joinedDF = dfBestTeams.join(filteredTeamsDF, ['TEAM_ID'])
joinedDF.show()

+---------+----------+--------+--------+------------+--------------+
|LEAGUE_ID|   TEAM_ID|MIN_YEAR|MAX_YEAR|ABBREVIATION|     HEADCOACH|
+---------+----------+--------+--------+------------+--------------+
|       00|1610612737|    1949|    2019|         ATL|  Lloyd Pierce|
|       00|1610612738|    1946|    2019|         BOS|  Brad Stevens|
|       00|1610612740|    2002|    2019|         NOP|  Alvin Gentry|
|       00|1610612741|    1966|    2019|         CHI|    Jim Boylen|
|       00|1610612742|    1980|    2019|         DAL| Rick Carlisle|
|       00|1610612743|    1976|    2019|         DEN|Michael Malone|
|       00|1610612745|    1967|    2019|         HOU| Mike D'Antoni|
|       00|1610612746|    1970|    2019|         LAC|    Doc Rivers|
|       00|1610612747|    1948|    2019|         LAL|   Frank Vogel|
|       00|1610612748|    1988|    2019|         MIA|Erik Spoelstra|
+---------+----------+--------+--------+------------+--------------+
only showing top 10 rows

+-------

In [80]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "temp_de2023_2124849"  
spark.conf.set('temporaryGcsBucket', bucket)

# Saving the data to BigQuery
joinedDF.write.format('bigquery') \
  .option('table', 'dataengineeringcourse2023.Output_processing_pipeline.bestTeamsByConference') \
  .mode("overwrite") \
  .save()